<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 2 - Preparación para la descarga de playlists</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---
<br>

<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>


* [1. Introducción](#section1)
* [2. Lectura de resultados](#section2)
* [3. Filtrado de resultados](#section3)
    * [3.1. Filtrado de identificadores repetidos](#section31)
    * [3.2. Filtrado por número de pistas](#section32)
    * [3.3. Filtrado por títulos](#section33)
* [4. Almacenamiento de identificadores](#section4)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
#!pip install pandas

In [3]:
import os
import pandas as pd
import random
import shutil

from tqdm.notebook import tqdm as tqdm_nb

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>

<br>

Tras haber realizado varias búsquedas empleando la [WebAPI de Spotify](https://developer.spotify.com/documentation/web-api/), hemos obtenido un conjunto de playlists con la siguiente información:

* _Identificador_ (**id**)
* _Título_ (**name**)
* *Número de pistas* (**num_tracks**)

<br>

Con esta información no podemos realizar un filtrado completo para decidir con qué listas nos vamos a quedar, pero si podemos emplearla para realizar varios descartes:

* Playlists repetidas (mismo identificador).
* Playlists que tienen menos de 5 pistas y más de 250 pistas.
* Playlists con títulos que contienen menos de 2 caracteres y más 50 caracteres (sin contar espacios).
* Playlists cuyos títulos contienen más de 9 palabras.

<br>

Una vez que hayamos eliminado aquellas listas que no nos interesan, vamos a quedarnos con los identificadores y los almacenaremos en varios ficheros que contengan el mismo número de elementos. Esto se hace para poder realizar la descarga de forma escalonada y poder utilizar varios procesos y/o máquinas a la vez, con lo cual se reduce el tiempo de obtención.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Lectura de resultados</font>
<br>

En este apartado vamos a leer los _DataSets_ que hemos obtenido en la libreta anterior, [_Búsqueda de Playlists_](01-BusquedaPlaylists.ipynb), y unirlos en un único [_DataFrame_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) de `pandas` para estudiarlos.

In [4]:
# Variables globales

# Directorio donde se encuentran los sets que deseamos leer
SETS_PATH = 'sets' 

# Directorio donde se almacenan los sets generados
PLS_ID_PATH = 'pls_id_sets'

# Directorio que emplearemos para guardar copias de seguridad.
BACKUP_PATH = 'backup'

In [5]:
# Crea un DataFrame a partir de los DataSets contenidos
# en el directorio indicado
def create_dataframe(sets_path):
    all_files = []

    for file_name in os.listdir(sets_path):
        if 'set-' in file_name:
            file_path = os.path.join(sets_path, file_name)
            all_files.append(file_path)
            
    list_df_files = (pd.read_csv(f,index_col=None,header=None) for f in all_files)

    df_concat = pd.concat(list_df_files, ignore_index=True, sort=False)
    df_concat.columns = ['id','name','tracks']

    # Elimina los sets del directorio
    for file_name in all_files:
        os.remove(file_name)         
    
    # Borramos el directorio que contenía los sets
    os.rmdir(sets_path)
    
    return df_concat

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Aunque hemos eliminado el directorio `SETS_PATH` y todo su contenido, disponemos de una copia de dicho contenido en la carpeta establecida en `BACKUP_PATH`.
</div>

<br>

In [6]:
df_playlists = create_dataframe(SETS_PATH)
print(f'Tamaño df_playlists: {len(df_playlists)}')

Tamaño df_playlists: 15797992


In [7]:
df_playlists.head()

,id,name,tracks
0,2MBSSLDiiOOtxJb2bVjO56,Keep It Mello,47
1,2hb42Q1VZi86YLktFHRiCn,Keep Flowing,161
2,6nuo9FYCAGV0WalhCluicG,Keep It R&B,58
3,5oxZIYU1L9N1CczN0C4JkM,Instrumental Pop Covers,401
4,1KPF87wrirpAWD0E7T24Lh,YOU SPIN ME RIGHT ROUND BABY RIGHT ROUND,61


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Filtrado de resultados</font>
<br>

Como hemos comentado anteriormente, vamos a intentar eliminar parte de los identificadores que hemos obtenido con el objetivo de reducir las playlists a descargar.


<a id="section31"></a>
### <font color="#B20033">3.1 - Filtrado de identificadores repetidos</font>

<br>

Eliminamos las playlists que figuran más de una vez (contienen el mismo __id__):

In [8]:
df_playlists.drop_duplicates(subset='id',inplace=True)
print(f'Tamaño df_playlists (sin ids duplicados): {len(df_playlists)}')

Tamaño df_playlists (sin ids duplicados): 12083093


Establecemos la columna __id__ como índice del DataFrame:

In [9]:
df_playlists.set_index('id',inplace=True)
df_playlists.head()

,name,tracks
id,,
2MBSSLDiiOOtxJb2bVjO56,Keep It Mello,47
2hb42Q1VZi86YLktFHRiCn,Keep Flowing,161
6nuo9FYCAGV0WalhCluicG,Keep It R&B,58
5oxZIYU1L9N1CczN0C4JkM,Instrumental Pop Covers,401
1KPF87wrirpAWD0E7T24Lh,YOU SPIN ME RIGHT ROUND BABY RIGHT ROUND,61


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section32"></a>
### <font color="#B20033">3.2 - Filtrado por número de pistas</font>

<br>

Nos quedamos con aquellas playlists que contienen entre 5 y 250 pistas:

In [10]:
df_playlists.drop(df_playlists[df_playlists['tracks'] > 250].index,inplace=True)
df_playlists.drop(df_playlists[df_playlists['tracks'] < 5].index,inplace=True)
print(f'Tamaño df_playlists (con filtrado de pistas): {len(df_playlists)}')

Tamaño df_playlists (con filtrado de pistas): 10548749


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section33"></a>
### <font color="#B20033">3.3 - Filtrado por títulos</font>

<br>

Eliminamos las playlists con títulos de más de 50 caracteres y menos de 2:

In [11]:
large_names_ids = df_playlists[(df_playlists.name.str.len()-df_playlists.name.str.count(' ')) > 50].index
sort_names_ids = df_playlists[(df_playlists.name.str.len()-df_playlists.name.str.count(' ')) < 2].index
df_playlists.drop(large_names_ids,inplace=True)
df_playlists.drop(sort_names_ids,inplace=True)
print(f'Tamaño df_playlists (con filtrado de títulos): {len(df_playlists)}')

Tamaño df_playlists (con filtrado de títulos): 10436758


<br>

Eliminamos las playlist con títulos de más de 9 palabras:

In [12]:
large_names_ids = df_playlists[df_playlists.name.str.count(' ') > 9].index
df_playlists.drop(large_names_ids,inplace=True)
print(f'Tamaño df_playlists (con filtrado por número de palabras): {len(df_playlists)}')

Tamaño df_playlists (con filtrado por número de palabras): 10394271


<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
**Importante**: En la libreta __[_Filtrado de Playlists_](04-FiltradoPlaylists.ipynb)__, volveremos a repetir estos dos últimos filtros que hemos empleado por los siguientes motivos:

* El usuario puede haber modificado el título de la playlists tras realizar nuestro proceso de búsqueda, de tal forma que cuando vayamos a obtener la playlist figure un nuevo título.


* Habrá pistas pertenecientes a una playlist que no están disponibles cuando descarguemos su información. Por lo tanto, el número de pistas que contenga puede variar.
    * *Por ejemplo*: Puede que exista una pista que haya dejado de estar disponible en Spotify tiempo después de que el usuario crease la playlist.
</div>


<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Almacenamiento de identificadores</font>

<br>

El último paso que vamos a realizar es el de almacenar en un directorio los identificadores de las playlists que hemos obtenido tras realizar el filtrado.

En primer lugar, obtenemos los valores de la columna __id__, los almacenamos en una lista llamada _ids_ y los mezclamos:

In [13]:
# Obtenemos todos los identificadores del DataFrame
ids = df_playlists.index.to_list()

random.shuffle(ids)

A continuación, dividiremos la lista en bloques (*chunks*) de 1000 identificadores:

In [14]:
# Establecemos el tamaño de bloque
chunk_len = 1000

# Almacenamos una lista con los bloques que hemos creado 
chunks = [ids[x:x+chunk_len] for x in range(0, len(ids), chunk_len)]

print(f"Bloques generados: {len(chunks)}")

Bloques generados: 10395


Una vez creados los bloques, procedemos a almacenarlos en ficheros:

In [15]:
if not os.path.exists(PLS_ID_PATH):
    os.makedirs(PLS_ID_PATH)
    
num = 0
for chunk in tqdm_nb(chunks):
    with open('{}/pls-id-set_{:05d}.txt'.format(PLS_ID_PATH,num), 'w', encoding='utf-8') as f:
        for item in chunk:
            f.write("%s\n" % item)
        num += 1

  0%|          | 0/10395 [00:00<?, ?it/s]

Por último, guardamos una copia de los sets generados en un fichero comprimido y lo movemos al directorio establecido en la variable `BACKUP_PATH`:

In [16]:
zipfile_name = "pls_id_sets"

shutil.make_archive(zipfile_name, 'zip', PLS_ID_PATH)
shutil.move(f'{zipfile_name}.zip', BACKUP_PATH)

'backup/pls_id_sets.zip'

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>